In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agent-Projects")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agent-Projects


In [3]:
import os

if not os.path.exists("tmp"):
    os.mkdir("tmp")

In [4]:
# FileManagementToolkit을 가져옵니다. 이 도구는 파일 관리 작업을 수행하는 데 사용됩니다.
from langchain_community.agent_toolkits import FileManagementToolkit

# 'tmp'라는 이름의 디렉토리를 작업 디렉토리로 설정합니다.
working_directory = "tmp"

# FileManagementToolkit 객체를 생성합니다.
# root_dir 매개변수에 작업 디렉토리를 지정하여 모든 파일 작업이 이 디렉토리 내에서 이루어지도록 합니다.
toolkit = FileManagementToolkit(root_dir=str(working_directory))

# toolkit.get_tools() 메서드를 호출하여 사용 가능한 모든 파일 관리 도구를 가져옵니다.
# 이 도구들은 파일 복사, 삭제, 검색, 이동, 읽기, 쓰기, 디렉토리 목록 조회 등의 기능을 제공합니다.
available_tools = toolkit.get_tools()

# 사용 가능한 도구들의 이름을 출력합니다.
print("[사용 가능한 파일 관리 도구들]")
for tool in available_tools:
    print(f"- {tool.name}: {tool.description}")

[사용 가능한 파일 관리 도구들]
- copy_file: Create a copy of a file in a specified location
- file_delete: Delete a file
- file_search: Recursively search for files in a subdirectory that match the regex pattern
- move_file: Move or rename a file from one location to another
- read_file: Read file from disk
- write_file: Write file to disk
- list_directory: List files and directories in a specified folder


In [5]:
# 도구 중 일부만 지정하여 선택하는 것도 가능합니다
tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=["read_file", "file_delete", "write_file", "list_directory"],
).get_tools()
tools

[ReadFileTool(root_dir='tmp'),
 DeleteFileTool(root_dir='tmp'),
 WriteFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp')]

In [8]:
read_tool, delete_tool, write_tool, list_tool = tools

# 파일 쓰기
write_tool.invoke({"file_path": "example.txt", "text": "Hello World!"})

'File written successfully to example.txt.'

In [ ]:
# 파일 삭제
# print(delete_tool.invoke({"file_path": "example.txt"}))

File deleted successfully: example.txt.


In [9]:
# 필요한 모듈과 클래스를 임포트합니다.
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews


# 최신 뉴스 검색 도구를 정의합니다.
@tool
def search_news(keyword: str, k: int = 5) -> List[Dict[str, str]]:
    """Look up latest news"""
    # GoogleNews 객체를 생성합니다.
    news_tool = GoogleNews()
    # 최신 뉴스를 검색하고 결과를 반환합니다. keyword는 키워드, k는 반환할 뉴스 항목의 수입니다.
    return news_tool.search_by_keyword(keyword=keyword,k=k)


# FileManagementToolkit을 사용하여 파일 관리 도구들을 가져옵니다.
tools = FileManagementToolkit(
    root_dir=str(working_directory),
).get_tools()

# 최신 뉴스 검색 도구를 tools 리스트에 추가합니다.
tools.append(search_news)

# 모든 도구들이 포함된 tools 리스트를 출력합니다.
tools

[CopyFileTool(root_dir='tmp'),
 DeleteFileTool(root_dir='tmp'),
 FileSearchTool(root_dir='tmp'),
 MoveFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 WriteFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp'),
 StructuredTool(name='search_news', description='Look up latest news', args_schema=<class 'langchain_core.utils.pydantic.search_news'>, func=<function search_news at 0x0000021549C29080>)]

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

# session_id 를 저장할 딕셔너리 생성
store = {}

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool to find news by keyword. "
            "Make sure to use the `file_management` tool to manage files. ",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini")

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

agent_stream_parser = AgentStreamParser()

In [13]:
result = agent_with_chat_history.stream(
    {
        "input": """게임에 대한 긍정적인 뉴스에 대해 검색하고, 각 뉴스의 제목과 url을 example.txt에 넣어줘.
        제목과 url을 넣을때 제목 밑에 url이 올 수 있도록해줘."""
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: search_news
keyword: 게임
k: 5
Log: 
Invoking: `search_news` with `{'keyword': '게임', 'k': 5}`



[관찰 내용]
Observation: [{'url': 'https://news.google.com/rss/articles/CBMieEFVX3lxTE5GSWFWbkZQTWg5cXlaWDc3R3hnemw3bURQWUJZV203NkRqaDhhVm9Jd21aSjAtVzlaRmx1MVZKaGFfSjVSNlkyMVNfOUVZY2E4RFNZclZMTlVCY0hqeUdqMTJ6LW1kU2ZqWWttdFkxc01PY18yWEhNWtIBjAFBVV95cUxNVmpkMkhHQTREaV9zU2lna3pVWTJoUmRjelgwMWFLT1JxOGZXVmtGYkhWNGhQZWI3X1ROVDF0XzBjdkpNWFRGeUZWVFJ1eHN1djRWYlF0S0FNR2NEUDVXSUwzRGNpZEVIT1g4MnlvaDVxOXRqc1BuWWF4a3h2NXNYS0tSbFliX1JDSzgzRA?oc=5', 'content': '유명 게임사 3곳, 아이템 확률 속여 총 2250만원 과태료 - 조선일보'}, {'url': 'https://news.google.com/rss/articles/CBMiT0FVX3lxTE5oMlVlS2taZ2ZZSTNkUHB3WlJfdW9NbGg3SUFiaUdhRmVzSUhPOGloRWpBbGd6SVYwdEl0T2NPeGllbk1ZRy1tSklOLVpwbDg?oc=5', 'content': '확률형 아이템 허위 광고 또 걸렸다…게임회사 3곳에 과태료 2천250만원 - v.daum.net'}, {'url': 'https://news.google.com/rss/articles/CBMiWEFVX3lxTFB0YjdhUnJVR1FUd252aGxEYWM2VV9acVFOWEZVaUYzbTNYMWU1cVRha0h5eW5zWWU3Y0F5T0NqZGVWUVZmc0lRQjg1OWFL